In [ ]:
import polars as pl
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pl.read_csv("../../Data/Titanic/train.csv")

In [112]:
def standardize(column):
    return (column - column.mean()) / column.std()

df = df.with_columns(
    [
        pl.when(pl.col("Sex")=="male").then(pl.lit(1)).otherwise(pl.lit(0)).alias("Sex"),
        pl.col("Age").fill_null(pl.col("Age").median()),
        pl.col("Fare").log1p().alias("Fare"),
    ]
).to_dummies(
    "Embarked"
).drop(
    [
        "PassengerId",
        "Name",
        "Ticket",
        "Cabin",
        "Embarked_S",
        "Embarked_null",
    ]
)

df = df.with_columns(
    [
        standardize(pl.col("Age")).alias("Age"),
        standardize(pl.col("Fare")).alias("Fare")
    ]
)

In [113]:
df.head()

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q
i64,i64,i32,f64,i64,i64,f64,u8,u8
0,3,1,-0.565419,1,0,-0.879247,0,0
1,1,0,0.663488,1,0,1.360456,1,0
1,3,0,-0.258192,0,0,-0.798092,0,0
1,1,0,0.433068,1,0,1.061442,0,0
0,3,1,0.433068,0,0,-0.783739,0,0


In [114]:
df.write_csv("../Data/baseDataframe.csv")